# Welcome to the Lab 🥼🧪

## Gross Yield and Median Rental Price

Let's do a deep dive on one specific rental market, and analyze how rental profits, as represented by gross yield could be related to rental prices.

At the market level, gross yield is calculated by dividing the annual median rental income by its median new listings for sale price. Gross yiled is often used by landlords (both institutional and non-institutional) to calculate the yearly return on a rental property without accounting for expenses. It is used as one input to understand the potential value of a rental property.

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/gross_yield_vs_rental_price.ipynb)

## Table of Contents

### [Rental Metrics](#metrics)
- Gross yield by month in Atlanta
- Median Rental price by month

### [Visualize the results for Analysis](#chart)
- Visualizing the monthly gross yield and median rental prices
- What's the relationship between gross yields and rents in Atlanta?

We will be creating the following chart: 

![Chart1](assets/gross_yield_and_rent_price.png)

In [ ]:
# Environment setup
import os
import sys
import subprocess
from datetime import datetime

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [ ]:
import parcllabs
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
client = ParclLabsClient(api_key=api_key)

In [ ]:
# we are looking at homes in Atlanta, where institutional activity is high, so we look for the name and the type of market. We can use state and location parameters to get the correct city
results = client.search_markets.retrieve(
    query='Atlanta',
    location_type='CITY',
    state_abbreviation='GA',
    as_dataframe=True
)

results.head()

In [ ]:
# we see that the the only result is the city of Atlanta, so we will use the parcl id of 5384169
analysis_parcl_id = results.iloc[0]['parcl_id']
name = results.iloc[0]['name']

# set default save directory
save_dir = './graphics'
new_dir_name = f"{analysis_parcl_id}-{name}-{datetime.today().date()}"
output_dir = os.path.join(save_dir, new_dir_name)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"In this analysis, we are studying: {name} which has parcl id: {analysis_parcl_id}. Saving graphics to: {save_dir}")

<a id='metrics'></a>
#### Rental Metrics

Let's start off this analysis by pulling in gross yields and median monthly rents for Atlanta, and then joining together the dataframes for analysis.

In [ ]:
# we call the specific groos yield endpoint, specifying our parcl id and ALL_PROPERTIES property type
gross_yield = client.rental_market_metrics_gross_yield.retrieve(
    parcl_id=analysis_parcl_id,
    property_type='ALL_PROPERTIES',
    params={
        'limit': 100
    },
    as_dataframe=True # set to true to default to a pandas dataframe for easier manipulation
)

gross_yield.head()

In [ ]:
# Next we query the event prices in Atlanta over the same time frame for all property types. 
rental_prices = client.market_metrics_housing_event_prices.retrieve(
    parcl_id=analysis_parcl_id,
    property_type='ALL_PROPERTIES',
    params={
        'limit': 100
    },
    as_dataframe=True # set to true to default to a pandas dataframe for easier manipulation
)

rental_prices.head()

In [ ]:
#Next you can join the rental prices to the gross yield dataframe to have one clean df for analysis. Make sure to join on both parcl id and date

gy_analysis = pd.merge(gross_yield, rental_prices[['parcl_id', 'date', 'price_median_new_rental_listings']], on=['parcl_id', 'date'])
gy_analysis.head()

<a id='chart'></a>
### Visualize the results for Analysis

Let's review rental prices, and gross yields on the same chart to understand the relationship.

In [ ]:
# set some constants for the analysis, such as the logo for the chart, we have a white and blue version
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white+(1).svg'
}

# set charting constants
labs_logo_dict = dict(
        source=labs_logo_lookup['white'],
        xref="paper",
        yref="paper",
        x=0.5,  # Centering the logo below the title
        y=1.02,  # Adjust this value to position the logo just below the title
        sizex=0.15, 
        sizey=0.15,
        xanchor="center",
        yanchor="bottom"
)

# define image dimentions
media_img_size_lookup = {
    'X': {
        'width': 1600,
        'height': 900
    }
}

# optimize the visual for the platform in this case X
PLATFORM = 'X'

# set image sizes
IMG_WIDTH = media_img_size_lookup[PLATFORM]['width']
IMG_HEIGHT = media_img_size_lookup[PLATFORM]['height']

# plotting title settings
PLOT_TITLE_SETTINGS = {
        'text': 'Gross Yield and Median Rental Price for Atlanta',
        'y':0.97,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }

In [ ]:
# Create figure with secondary y-axis
fig = go.Figure()

# Add the first line
fig.add_trace(
    go.Scatter(
    x = gy_analysis['date'],
    y = gy_analysis['pct_gross_yield'],
    name='Gross Yield',
    line=dict(color='#448CF2', width=3),
    mode='lines+markers',
    marker=dict(color='#448CF2', size=5)
    ),
)

# Add the second line
fig.add_trace(
    go.Scatter(
    x = gy_analysis['date'],
    y = gy_analysis['price_median_new_rental_listings'],
    name='Median Rent',
    line=dict(color='#FFFFFF', width=3),
    mode='lines+markers',
    marker=dict(color='#FFFFFF', size=5),
    yaxis="y2"
    ),
)

# Add the logo as a layout_image
fig.add_layout_image(
    labs_logo_dict
)

# format our figure
fig.update_layout(
    xaxis=dict(title='Date'),
    yaxis=dict(title='Gross Yield (%)', side='left', showgrid=False),
    yaxis2=dict(title='Median Rental Price ($)', side='right', overlaying='y', showgrid=False, anchor="x", tickformat=',.0f'),
    margin=dict(b=100),
    width=IMG_WIDTH,
    height=IMG_HEIGHT,
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top',
        title='Gross Yield x Rental Price'
    ),
    title=PLOT_TITLE_SETTINGS,
    plot_bgcolor='#080D16',
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
)

# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# save fig
fig.write_image(os.path.join(output_dir, 'gross_yield_and_rent_price.png'), width=IMG_WIDTH, height=IMG_HEIGHT)
# Show the plot
fig.show()

 We can see that in the second half of 2020 and through 2021 median rents in the city skyrocketed and have since maintained a new normal. Additional gross yields remained high through 2021. Since then, however, gross yields have consistently declined, although still maintaining higher than 5% returns for owners, while rents have remained high - likely due to the rising price of homes for sale.